# 준비
라이브러리, 파라미터 세팅

In [18]:
import pandas as pd
import numpy as np
import os
import re

In [19]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'

TEST_SIZE = 0.2
RANDOM_SEED = 42

# 데이터 불러오기

In [20]:
# 테스트(캐글) 데이터 로드
test_data = pd.read_csv(DATA_IN_PATH + 'en_data_modified.csv')
test_data.drop('i_dialog', axis=1, inplace=True)
test_data.drop('i_utterance', axis=1, inplace=True)
test_data.drop('speaker', axis=1, inplace=True)
test_data.columns = ['Id','utterance']  # 컬럼명 변경
test_data.head()

,Id,utterance
0,0,"Alright, whadyou do with him?"
1,1,Oh! You're awake!
2,2,Then you gotta come clean with Ma! This is not...
3,3,"Yeah, but this is"
4,4,I don't wanna hear it! Now go to my room!


In [21]:
# ELECTRA(electra-small-generator 모델) 예측 데이터 로드 1
predict_electra_small1 = pd.read_csv(DATA_OUT_PATH + 'FRIENDS_ELECTRA(01).csv')
predict_electra_small1.head()

,Id,Predicted
0,0,neutral
1,1,surprise
2,2,anger
3,3,neutral
4,4,anger


In [22]:
# ELECTRA(electra-small-generator 모델) 예측 데이터 로드 2
predict_electra_small2 = pd.read_csv(DATA_OUT_PATH + 'FRIENDS_ELECTRA(02).csv')
predict_electra_small2.head()

,Id,Predicted
0,0,neutral
1,1,surprise
2,2,anger
3,3,neutral
4,4,non-neutral


In [23]:
# ELECTRA(electra-base-generator 모델) 예측 데이터 로드
predict_electra_base = pd.read_csv(DATA_OUT_PATH + 'FRIENDS_ELECTRA(05).csv')
predict_electra_base.head()

,Id,Predicted
0,0,neutral
1,1,surprise
2,2,anger
3,3,neutral
4,4,joy


In [24]:
# ELECTRA(electra-large-generator 모델) 예측 데이터 로드 1
predict_electra_large1 = pd.read_csv(DATA_OUT_PATH + 'FRIENDS_ELECTRA(10).csv')
predict_electra_large1.head()

,Id,Predicted
0,0,neutral
1,1,surprise
2,2,anger
3,3,neutral
4,4,anger


In [25]:
# ELECTRA(electra-large-generator 모델) 예측 데이터 로드 2
predict_electra_large2 = pd.read_csv(DATA_OUT_PATH + 'FRIENDS_ELECTRA(15).csv')
predict_electra_large2.head()

,Id,Predicted
0,0,neutral
1,1,surprise
2,2,non-neutral
3,3,neutral
4,4,non-neutral


# 기본 모델 (Baseline)
### 모델 앙상블(Majority Voting) 방식

In [26]:
# 테스트 데이터의 id 부분을 리스트 처리
test_id = list(test_data['Id'])
len(test_id)

1623

In [27]:
# predict_electra_small1 데이터의 Predicted 부분을 리스트 처리
electra_predicted_small1 = list(predict_electra_small1['Predicted'])
len(electra_predicted_small1)

1623

In [28]:
# predict_electra_small2 데이터의 Predicted 부분을 리스트 처리
electra_predicted_small2 = list(predict_electra_small2['Predicted'])
len(electra_predicted_small2)

1623

In [29]:
# predict_electra_base 데이터의 Predicted 부분을 리스트 처리
electra_predicted_base = list(predict_electra_base['Predicted'])
len(electra_predicted_base)

1623

In [30]:
# predict_electra_large1 데이터의 Predicted 부분을 리스트 처리
electra_predicted_large1 = list(predict_electra_large['Predicted'])
len(electra_predicted_large)

1623

In [31]:
# predict_electra_large1 데이터의 Predicted 부분을 리스트 처리
electra_predicted_large2 = list(predict_electra_large['Predicted'])
len(electra_predicted_large2)

1623

In [32]:
# # 판다스 데이터프레임 통해 데이터 구성하여 output에 투입
# result = pd.DataFrame( data={"Id": test_id, "ELECTRA_SMALL1": electra_predicted_small1 \
#                              ,"ELECTRA_SMALL2": electra_predicted_small2 \
#                              ,"ELECTRA_BASE": electra_predicted_base \
#                              ,"ELECTRA_LARGE": electra_predicted_large \
#                             } )
# result.head(20)

In [33]:
# 판다스 데이터프레임 통해 데이터 구성하여 output에 투입
result = pd.DataFrame( data={"Id": test_id, "ELECTRA_SMALL1": electra_predicted_small1 \
                             ,"ELECTRA_SMALL2": electra_predicted_small2 \
                             ,"ELECTRA_LARGE1": electra_predicted_large1 \
                             ,"ELECTRA_LARGE2": electra_predicted_large2 \
                            } )
result.head(20)

,Id,ELECTRA_SMALL1,ELECTRA_SMALL2,ELECTRA_LARGE1,ELECTRA_LARGE2
0,0,neutral,neutral,neutral,neutral
1,1,surprise,surprise,surprise,surprise
2,2,anger,anger,anger,anger
3,3,neutral,neutral,neutral,neutral
4,4,anger,non-neutral,anger,anger
5,5,non-neutral,non-neutral,anger,anger
6,6,neutral,neutral,neutral,neutral
7,7,neutral,neutral,non-neutral,non-neutral
8,8,surprise,surprise,surprise,surprise
9,9,neutral,neutral,neutral,neutral


In [34]:
ensemble_result = []

for i in range(len(result)):  # Id 별
    cnt_neutral = 0
    cnt_non_neutral = 0
    cnt_joy = 0
    cnt_sadness = 0
    cnt_fear = 0
    cnt_anger = 0
    cnt_surprise = 0
    cnt_disgust = 0
    
    for j in range(1,5):  # 모델 별
        if result.iloc[i,j] == 'neutral':
            cnt_neutral += 1
        if result.iloc[i,j] == 'non-neutral':
            cnt_non_neutral += 1
        if result.iloc[i,j] == 'joy':
            cnt_joy += 1
        if result.iloc[i,j] == 'sadness':
            cnt_sadness += 1
        if result.iloc[i,j] == 'fear':
            cnt_fear += 1
        if result.iloc[i,j] == 'anger':
            cnt_anger += 1
        if result.iloc[i,j] == 'surprise':
            cnt_surprise += 1
        if result.iloc[i,j] == 'disgust':
            cnt_disgust += 1            
    
    print(i, 'neutral:', cnt_neutral \
          ,  'non-neutral:', cnt_non_neutral \
          ,  'joy:', cnt_joy \
          ,  'sadness:', cnt_sadness \
          ,  'fear:', cnt_fear \
          ,  'anger:', cnt_anger \
          ,  'surprise:', cnt_surprise \
          ,  'disgust:', cnt_disgust \
         )

      # 0.56350
#     if cnt_neutral >= 2:  ensemble_result.append('neutral')
#     elif cnt_non_neutral >= 2:  ensemble_result.append('non-neutral')
#     elif cnt_joy >= 2:  ensemble_result.append('joy')
#     elif cnt_sadness >= 2:  ensemble_result.append('sadness')
#     elif cnt_fear >= 2:  ensemble_result.append('fear')
#     elif cnt_anger >= 2:  ensemble_result.append('anger')
#     elif cnt_surprise >= 2:  ensemble_result.append('surprise')
#     elif cnt_disgust >= 2:  ensemble_result.append('disgust')
#     else:  ensemble_result.append('neutral')
    
    # 0.56226
#     if cnt_fear >= 2:  ensemble_result.append('fear')
#     elif cnt_disgust >= 2:  ensemble_result.append('disgust')
#     elif cnt_sadness >= 2:  ensemble_result.append('sadness')
#     elif cnt_anger >= 2:  ensemble_result.append('anger')
#     elif cnt_surprise >= 2:  ensemble_result.append('surprise')
#     elif cnt_joy >= 2:  ensemble_result.append('joy')
#     elif cnt_neutral >= 2:  ensemble_result.append('neutral')
#     elif cnt_non_neutral >= 2:  ensemble_result.append('non-neutral')
#     else:  ensemble_result.append('neutral')
    
    # 0.56473
#     if cnt_neutral >= 2:  ensemble_result.append('neutral')
#     elif cnt_non_neutral >= 2:  ensemble_result.append('non-neutral')
#     elif cnt_joy >= 2:  ensemble_result.append('joy')
#     elif cnt_surprise >= 2:  ensemble_result.append('surprise')
#     elif cnt_anger >= 2:  ensemble_result.append('anger')
#     elif cnt_sadness >= 2:  ensemble_result.append('sadness')
#     elif cnt_disgust >= 2:  ensemble_result.append('disgust')
#     elif cnt_fear >= 2:  ensemble_result.append('fear')
#     else:  ensemble_result.append('neutral')

    # 0.56596 => 학습 데이터에서 빈도가 가장 높은 것에서 낮은 순으로 정렬
    if cnt_neutral >= 2:  ensemble_result.append('neutral')
    elif cnt_non_neutral >= 2:  ensemble_result.append('non-neutral')
    elif cnt_joy >= 2:  ensemble_result.append('joy')
    elif cnt_surprise >= 2:  ensemble_result.append('surprise')
    elif cnt_anger >= 2:  ensemble_result.append('anger')
    elif cnt_sadness >= 2:  ensemble_result.append('sadness')
    elif cnt_disgust >= 2:  ensemble_result.append('disgust')
    elif cnt_fear >= 2:  ensemble_result.append('fear')
    else:  ensemble_result.append('non-neutral')  # 특정 감정으로 단정할 수 없으며 중립으로 보기 애매하다고 세팅
    
result["Ensemble_Result"] = ensemble_result

0 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1 neutral: 0 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 4 disgust: 0
2 neutral: 0 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 4 surprise: 0 disgust: 0
3 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
4 neutral: 0 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 3 surprise: 0 disgust: 0
5 neutral: 0 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 2 surprise: 0 disgust: 0
6 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
7 neutral: 2 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
8 neutral: 0 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 4 disgust: 0
9 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
10 neutral: 0 non-neutral: 4 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
11 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 

106 neutral: 3 non-neutral: 0 joy: 0 sadness: 1 fear: 0 anger: 0 surprise: 0 disgust: 0
107 neutral: 0 non-neutral: 3 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
108 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
109 neutral: 0 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 2 disgust: 0
110 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
111 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
112 neutral: 2 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
113 neutral: 2 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
114 neutral: 2 non-neutral: 0 joy: 2 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
115 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
116 neutral: 0 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 4 disgust: 0
117 neutral: 0 non-neutral: 3 jo

212 neutral: 0 non-neutral: 4 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
213 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
214 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
215 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
216 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
217 neutral: 2 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
218 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
219 neutral: 1 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 2 surprise: 0 disgust: 0
220 neutral: 0 non-neutral: 3 joy: 1 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
221 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
222 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
223 neutral: 4 non-neutral: 0 jo

316 neutral: 0 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 2 surprise: 0 disgust: 0
317 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
318 neutral: 0 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 4 disgust: 0
319 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
320 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
321 neutral: 0 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 4 disgust: 0
322 neutral: 0 non-neutral: 3 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
323 neutral: 2 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 2 disgust: 0
324 neutral: 0 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 4 disgust: 0
325 neutral: 0 non-neutral: 1 joy: 2 sadness: 0 fear: 0 anger: 0 surprise: 1 disgust: 0
326 neutral: 0 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 1 surprise: 2 disgust: 0
327 neutral: 4 non-neutral: 0 jo

420 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
421 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
422 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
423 neutral: 0 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 2 disgust: 0
424 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
425 neutral: 2 non-neutral: 0 joy: 2 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
426 neutral: 0 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 2 surprise: 0 disgust: 0
427 neutral: 0 non-neutral: 4 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
428 neutral: 0 non-neutral: 4 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
429 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
430 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
431 neutral: 4 non-neutral: 0 jo

...

1160 neutral: 0 non-neutral: 4 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1161 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1162 neutral: 0 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 2 surprise: 0 disgust: 0
1163 neutral: 0 non-neutral: 3 joy: 0 sadness: 1 fear: 0 anger: 0 surprise: 0 disgust: 0
1164 neutral: 0 non-neutral: 1 joy: 0 sadness: 3 fear: 0 anger: 0 surprise: 0 disgust: 0
1165 neutral: 3 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1166 neutral: 0 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 4 disgust: 0
1167 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1168 neutral: 0 non-neutral: 4 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1169 neutral: 0 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 4 disgust: 0
1170 neutral: 0 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 4 disgust: 0
1171 neutral: 0 non-n

1265 neutral: 0 non-neutral: 0 joy: 2 sadness: 0 fear: 0 anger: 0 surprise: 2 disgust: 0
1266 neutral: 0 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 4 disgust: 0
1267 neutral: 2 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1268 neutral: 0 non-neutral: 4 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1269 neutral: 0 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 4 disgust: 0
1270 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1271 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1272 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1273 neutral: 0 non-neutral: 0 joy: 4 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1274 neutral: 0 non-neutral: 0 joy: 4 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1275 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1276 neutral: 0 non-n

1373 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1374 neutral: 2 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1375 neutral: 0 non-neutral: 4 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1376 neutral: 0 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 4 disgust: 0
1377 neutral: 0 non-neutral: 0 joy: 4 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1378 neutral: 3 non-neutral: 1 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1379 neutral: 0 non-neutral: 4 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1380 neutral: 0 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 4 disgust: 0
1381 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1382 neutral: 2 non-neutral: 0 joy: 2 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1383 neutral: 1 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 3 disgust: 0
1384 neutral: 0 non-n

1481 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1482 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1483 neutral: 0 non-neutral: 3 joy: 1 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1484 neutral: 0 non-neutral: 4 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1485 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1486 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1487 neutral: 0 non-neutral: 0 joy: 4 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1488 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1489 neutral: 0 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 4 disgust: 0
1490 neutral: 0 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 4 disgust: 0
1491 neutral: 0 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 2 disgust: 0
1492 neutral: 0 non-n

1592 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1593 neutral: 0 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 2 disgust: 0
1594 neutral: 0 non-neutral: 4 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1595 neutral: 0 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 4 disgust: 0
1596 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1597 neutral: 0 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 4 disgust: 0
1598 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1599 neutral: 0 non-neutral: 2 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 2 disgust: 0
1600 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1601 neutral: 0 non-neutral: 4 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1602 neutral: 4 non-neutral: 0 joy: 0 sadness: 0 fear: 0 anger: 0 surprise: 0 disgust: 0
1603 neutral: 4 non-n

In [35]:
result.head(20)

,Id,ELECTRA_SMALL1,ELECTRA_SMALL2,ELECTRA_LARGE1,ELECTRA_LARGE2,Ensemble_Result
0,0,neutral,neutral,neutral,neutral,neutral
1,1,surprise,surprise,surprise,surprise,surprise
2,2,anger,anger,anger,anger,anger
3,3,neutral,neutral,neutral,neutral,neutral
4,4,anger,non-neutral,anger,anger,anger
5,5,non-neutral,non-neutral,anger,anger,non-neutral
6,6,neutral,neutral,neutral,neutral,neutral
7,7,neutral,neutral,non-neutral,non-neutral,neutral
8,8,surprise,surprise,surprise,surprise,surprise
9,9,neutral,neutral,neutral,neutral,neutral


In [36]:
output = result[['Id','Ensemble_Result']].copy()
output = output.rename({'Ensemble_Result':'Predicted'}, axis='columns')
output.head()

,Id,Predicted
0,0,neutral
1,1,surprise
2,2,anger
3,3,neutral
4,4,anger


In [37]:
# 해당 경로가 없으면 생성
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

# csv파일 생성
output.to_csv(DATA_OUT_PATH + "FRIENDS_Ensemble_ELECTRA.csv", index = False)  # 앙상블 결과, 캐글 제출 용도

### 캐글 제출 결과
**[2020.12.12]**<br>
ELECTRA_SMALL1 (0.56720),<br>
ELECTRA_SMALL2 (0.56720),<br>
ELECTRA_BASE (0.56596),<br>
ELECTRA_LARGE (0.56350)<br>
조합(4개) => 0.56596<br>

ELECTRA_SMALL1 (0.56720),<br>
ELECTRA_SMALL2 (0.56720),<br>
ELECTRA_BASE (0.56596),<br>
ELECTRA_LARGE (0.57336)<br>
조합(4개) => 0.57706<br>

ELECTRA_SMALL1 (0.56720),<br>
ELECTRA_SMALL2 (0.56720),<br>
ELECTRA_LARGE1 (0.57336),<br>
ELECTRA_LARGE2 (0.56966)<br>
조합(4개) => 0.57829 (Best!!)<br>

개별 모델 결과보다 성능 향상됨을 확인